In [ ]:
from preprocess_income_data import preprocess_data

X_train, X_test, X_val, y_train, y_val, y_test = preprocess_data()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC
import numpy as np
from const import N_SPLITS, RANDOM_STATE, SVM, SVM_ID

ALGORITHM_NAME = SVM
ALGORITHM_ID = SVM_ID

kf = KFold(n_splits=20, random_state=RANDOM_STATE, shuffle=True)
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

gamma = ["auto", "scale"]

steps = [(ALGORITHM_NAME, SVC(
    random_state=RANDOM_STATE,
    ))]


pipeline = Pipeline(steps)

parameters = {
    f"{ALGORITHM_NAME}__C": C,
    f"{ALGORITHM_NAME}__gamma": gamma,
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=kf, scoring="accuracy")

cv.fit(X_train, y_train)


In [ ]:
cv.best_score_

In [ ]:
from process_cv_results import process_cv_scores, process_best_params, get_classification_metrics, results_to_csv
accuracy_scores_df = process_cv_scores(cv, algorithm_id=ALGORITHM_ID)
accuracy_scores_df.head()


In [ ]:
best_params_df = process_best_params(
    cv, algorithm_id=ALGORITHM_ID, algorithm_name=ALGORITHM_NAME)
best_params_df.head()


In [ ]:
y_pred = cv.predict(X_test)
classification_metrics_df = get_classification_metrics(
    y_test, y_pred, cv, ALGORITHM_ID, ALGORITHM_NAME)

classification_metrics_df.head()


In [ ]:
dfs = [accuracy_scores_df, best_params_df, classification_metrics_df]

results_to_csv(dfs, ALGORITHM_ID)
